In [216]:
# Dependencies
import requests
import logging
import pandas as pd

In [217]:
# League Options
league_id = "564698"
year = ['2023','2022','2021','2020','2019','2018']

In [218]:
#Cookies for Private League Access
swid = "{823B7B7A-5AFC-4B2E-A22A-B2D690D236A8}"
espn_s2 = "AEBMf%2FfmL20PESZFBMdSRkR9HHsxOE%2FvAirc8HkQxtgX9tbG8vsxqa8Qm83LNJam6rJeMAWfmMVt3b2EuNYu7%2FATa0vER%2FTQIfoZRRg4ItNjN8MkU%2FBdNeTsOGei2ZJDgGX3zWxfxkysoQgRaVkikEHDHZtxlY9ZBtk3NrSUE%2BlaTWzbsfsEGlIgNzN4YG4QuUV7GxteDw1hUd%2Fb2sfdvew0ypoKcXrE51OQL5AgdfnH144uN1YmgFd3nNdBe%2FBIadjuJ2TZ7LAzAI7z1P6x7QhiG1Ah7RMTEzO7MDzoLdMODQ%3D%3D"

In [219]:
# Schedule data for Matchups
seasons = []
for yr in year:
    url = "https://lm-api-communication.fantasy.espn.com/apis/v3/games/ffl/seasons/"+yr+"/segments/0/leagues/564698"
    matchup_response = requests.get(url, cookies={"swid": swid,
                          "espn_s2": espn_s2},params={"view": "mMatchup"})
    matchup_json = matchup_response.json()
    df = [
    [
        game['matchupPeriodId'],
        game['home']['teamId'], 
        game['home'].get('totalPoints', 0),  # Using .get() to handle missing points
        game.get('away', {}).get('teamId', 'N/A'),  # Defaulting to 'N/A' if 'away' key is missing
        game.get('away', {}).get('totalPoints', 0)  # Defaulting to 0 if 'away' key or totalPoints is missing
    ]
    for game in matchup_json['schedule']
    ]
    df = pd.DataFrame(df, columns=['Week', 'HomeTeamId', 'HomeTotalPoints', 'AwayTeamId', 'AwayTotalPoints'])
    df['Type'] = ['Regular' if w<=14 else 'Playoff' for w in df['Week']]
    df['Year'] = yr
    seasons.append(df)

In [74]:
df_sch = pd.concat(seasons)

In [75]:
df_sch.to_csv('df_sch.csv')

In [76]:
# Team Data Names
seasons_tm = []
for yr in year:
    url = "https://lm-api-communication.fantasy.espn.com/apis/v3/games/ffl/seasons/"+yr+"/segments/0/leagues/564698"
    team_response = requests.get(url, cookies={"swid": swid,
                          "espn_s2": espn_s2},params={"view": "forTeamId"})
    team_json = team_response.json()
    tm = [
    [
        team['abbrev'],
        team['id'],
        team['owners']
    ]
    for team in team_json['teams']
    ]
    
    tm1 = pd.DataFrame(tm, columns=['Abbr', 'TeamID','OwnerID_2'])

    tm = [
    [
        team['displayName'],
        team['id'],
    ]
    for team in team_json['members']
    ]
    
    tm2 = pd.DataFrame(tm, columns=['displayName','OwnerID'])

    # Creating the Cartesian product directly
    dataFrameFull = pd.merge(tm1, tm2, how='cross')

    # Checking if OwnerID_2 is a substring of OwnerID
    dataFrameFull['match'] = dataFrameFull.apply(lambda x: x.OwnerID in x.OwnerID_2, axis=1)
    
    dataFrameFull['Year'] = yr
    seasons_tm.append(dataFrameFull[dataFrameFull['match']])

In [77]:
df_tm = pd.concat(seasons_tm)

In [78]:
df_tm.to_csv('df_tm.csv')

In [79]:
# Team Data Names
seasons_tm_1 = []
for yr in year:
    url = "https://lm-api-communication.fantasy.espn.com/apis/v3/games/ffl/seasons/"+yr+"/segments/0/leagues/564698"
    team_response = requests.get(url, cookies={"swid": swid,
                          "espn_s2": espn_s2},params={"view": "mTeam"})
    team_json = team_response.json()
    tm = [
    [
        team['displayName'],
        team['firstName'],
        team['lastName'],
        team['id']
    ]
    for team in team_json['members']
    ]
    
    tm1 = pd.DataFrame(tm, columns=['DisplayName', 'First','Last','OwnerID'])
    tm1['Year'] = yr
    seasons_tm_1.append(tm1)

In [80]:
df_tm1 = pd.concat(seasons_tm_1)

In [81]:
df_tm1.to_csv('df_tm1.csv')

In [82]:
#df_tm.merge(df_tm1).groupby(['TeamID'])['OwnerID'].nunique()

In [220]:
# Need this to identify Positions
slotcodes = {
    0 : 'QB', 1 : 'QB',
    2 : 'RB', 3 : 'RB',
    4 : 'WR', 5 : 'WR',
    6 : 'TE', 7 : 'TE',
    16: 'D/ST',
    17: 'K',
    20: 'Bench',
    21: 'IR',
    23: 'Flex'
}

In [84]:
def get_matchups(league_id, season, week, swid='', espn=''):
    ''' 
    Pull full JSON of matchup data from ESPN API for a particular week.
    '''
    
    url = 'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/'+str(season) + '/segments/0/leagues/' + str(league_id)+'?scoringPeriodId='+str(week)+ '&view=mMatchup&view=mMatchupScore' 

    r = requests.get(url,
                     cookies={"SWID": swid, "espn_s2": espn})
    return r.json()

def get_slates(json,season, week):
    '''
    Constructs week team slates with slotted position, 
    position, and points (actual and ESPN projected),
    given full matchup info (`get_matchups`)
    '''
    
    slates = {}
    slates_ls = []

    for team in d['teams']:
        slate = []
        for p in team['roster']['entries']:
            # get name
            if p.get('playerPoolEntry') and p['playerPoolEntry'].get('player'):
                name = p['playerPoolEntry']['player']['fullName']
            else:
                name = "Unknown Player"

            # get actual lineup slot
            slotid = p['lineupSlotId']
            slot = slotcodes[slotid]

            # get projected and actual scores
            act, proj = 0, 0
            if p.get('playerPoolEntry') and p['playerPoolEntry'].get('player'):
                for stat in p['playerPoolEntry']['player']['stats']:
                    if stat['scoringPeriodId'] != week:
                        continue
                    if stat['statSourceId'] == 0:
                        act = stat['appliedTotal']
                    elif stat['statSourceId'] == 1:
                        proj = stat['appliedTotal']
                    else:
                        print('Error')
            else:
                continue

            # get type of player
            pos = 'Unk'
            if p.get('playerPoolEntry') and p['playerPoolEntry'].get('player'):
                ess = p['playerPoolEntry']['player']['eligibleSlots']
                if 0 in ess: pos = 'QB'
                elif 2 in ess: pos = 'RB'
                elif 4 in ess: pos = 'WR'
                elif 6 in ess: pos = 'TE'
                elif 16 in ess: pos = 'D/ST'
                elif 17 in ess: pos = 'K'
            else:
                continue
            #teamid = team['id']
            
            
            slate.append([name, season, week, slotid, slot, pos, act, proj])

        slate = pd.DataFrame(slate, columns=['Name','Year', 'Week', 'SlotID', 'Slot', 'Pos', 'Actual', 'Proj'])
        #slates_ls.append(slate)
        slates[team['id']] = slate

    return slates

def compute_pts(slates, posns, struc):
    '''
    Given slates (`get_slates`), compute total roster points:
    actual, optimal, and using ESPN projections.
    
    Parameters
    --------------
    slates : `dict` of `DataFrames`
        (from `get_slates`)
    posns : `list`
        roster positions, e.g. ['QB', 'RB', 'WR', 'TE']
    struc : `list`
        slots per position, e.g. [1,2,2,1]
        
    * This is not flexible enough to handle "weird" leagues
    like 6 Flex slots with constraints on # total RB/WR
    
    Returns
    --------------
    `dict` of `dict`s with actual, ESPN, and optimal points
    '''
    
    data = {}
    for tmid, slate in slates.items():
        pts = {'opts': 0, 'epts': 0, 'apts': 0}

        # ACTUAL STARTERS
        pts['apts'] = slate.query('Slot not in ["Bench", "IR"]').filter(['Actual']).sum().values[0]

        # OPTIMAL and ESPNPROJ STARTERS
        for method, cat in [('Actual', 'opts'), ('Proj', 'epts')]:
            flex_candidates_act = []
            flex_candidates_proj = []
            fcp = []
            for pos, num in zip(posns, struc):
                # actual/projection points, sorted by the respective method
                sorted_slate = slate.query('Pos == @pos').sort_values(by=method, ascending=False)
                #print(sorted_slate.info())
                #print(method, pos,sorted_slate.iloc[:num].filter(['Actual']).sum().values[0])
                # Sum up the points for the starters in this position
                
                pts[cat] += sorted_slate.iloc[:num].filter(['Actual']).sum().values[0]
                

                # Add the next best as a flex candidate
                if pos in ['RB', 'WR', 'TE'] and len(sorted_slate) > num:
                    if method=="Proj":
                        
                        flex_candidates_proj.append(sorted_slate.iloc[num:])
                    else:
                        flex_candidates_act.extend(sorted_slate.iloc[num:].filter(['Actual']).values[:, 0])
                    #print(tmid, method, pos,flex_candidates_act,fcp)
            if flex_candidates_proj:
                fcp.extend(pd.concat(flex_candidates_proj).sort_values(by="Proj").iloc[num:].filter(['Actual']).values[:, 0])
                #print(fcp)
                top_flex = fcp[-num:]
                pts[cat] += sum(top_flex)
                #print(tmid, pos,method, num , sum(top_flex))
            # Add the best flex option to the total
            if flex_candidates_act:
                top_flex = sorted(flex_candidates_act, reverse=True)[:num]
                pts[cat] += sum(top_flex)
                
                #print(tmid, pos,method, num , sum(top_flex))
        
        data[tmid] = pts
        
    return data

#def get_teamnames(league_id, season, week, swid='', espn=''):
    #url = 'https://lm-api-communication.fantasy.espn.com/apis/v3/games/ffl/seasons/' + \
      #str(season) + '/segments/0/leagues/' + str(league_id)
    
    #r = requests.get(url + '?view=mTeam',
                  #params={'scoringPeriodId': week},
                  #cookies={"SWID": swid, "espn_s2": espn})
    #d = r.json()
    
    #tm_names = {tm['id']: tm['location'].strip() + ' ' + tm['nickname'].strip() \
               # for tm in d['teams']}
    
    #return tm_names

In [99]:
# I have to break this up because I have two different league structures with two flexs replacing one flex and a kicker
league_id = 564698

yrs = [2023,2022,2021]
posns = ['QB', 'RB', 'WR', 'Flex', 'TE', 'D/ST']
struc = [1,2,2,2,1,1,1]

data = {}
slates = {}
ds = {}
print('Week:', end=' ')
for y in yrs:
    data[y] = {}   # Initialize the year-specific dictionary in data
    slates[y] = {} # Initialize the year-specific dictionary in slates
    ds[y] = {}
    for week in range(1,18):
        print(week, y, end=' ')
        d      = get_matchups(league_id, y, week, swid=swid, espn=espn_s2)
        wslate = get_slates(d, y, week)
        wdata  = compute_pts(wslate, posns, struc)
        data[y][week] = wdata
        slates[y][week] = wslate
        ds[y][week] = d

Week: 1 2023 2 2023 3 2023 4 2023 5 2023 6 2023 7 2023 8 2023 9 2023 10 2023 11 2023 12 2023 13 2023 14 2023 15 2023 16 2023 17 2023 1 2022 2 2022 3 2022 4 2022 5 2022 6 2022 7 2022 8 2022 9 2022 10 2022 11 2022 12 2022 13 2022 14 2022 15 2022 16 2022 17 2022 1 2021 2 2021 3 2021 4 2021 5 2021 6 2021 7 2021 8 2021 9 2021 10 2021 11 2021 12 2021 13 2021 14 2021 15 2021 16 2021 17 2021 

In [96]:
#slates

In [101]:
# Getting lineups on a weekly basis. I did not use this but if you want to. You can format it this way. 
lineup_df = pd.DataFrame()

# Iterate through each year, week, and team DataFrame
for year, week_data in slates.items():
    for week, team_data in week_data.items():
        for team_id, df in team_data.items():
            # Add 'Year' and 'TeamID' columns to each DataFrame
            df['Year'] = year
            df['Week'] = week
            df['TeamID'] = team_id
            # Append to the combined DataFrame
            lineup_df = pd.concat([lineup_df, df], ignore_index=True)

# Optionally, reorder columns if necessary
lineup_df

,Name,Year,Week,SlotID,Slot,Pos,Actual,Proj,TeamID
0,Bijan Robinson,2023,1,2,RB,RB,18.3,0.000000,1
1,Garrett Wilson,2023,1,4,WR,WR,11.9,0.000000,1
2,Dameon Pierce,2023,1,2,RB,RB,6.8,0.000000,1
3,Amari Cooper,2023,1,4,WR,WR,5.2,0.000000,1
4,Alvin Kamara,2023,1,20,Bench,RB,0.0,0.000000,1
...,...,...,...,...,...,...,...,...,...
8412,Patriots D/ST,2021,17,16,D/ST,D/ST,15.0,9.615185,12
8413,Colts D/ST,2021,17,20,Bench,D/ST,9.0,8.750749,12
8414,Duke Johnson,2021,17,20,Bench,RB,8.2,7.137854,12
8415,Marquez Callaway,2021,17,20,Bench,WR,12.7,7.550020,12


In [102]:
# Optimal Points, Epts, Apts format into a Dataframe
df1 = pd.DataFrame.from_dict(
    {(year, week, team): stats
     for year, weeks in data.items()
     for week, teams in weeks.items()
     for team, stats in teams.items()},
    orient='index'
)

df1.reset_index(inplace=True)
df1.columns = ['Year', 'Week', 'TeamID', 'Opts', 'Epts', 'Apts']

df1

,Year,Week,TeamID,Opts,Epts,Apts
0,2023,1,1,134.4,90.3,107.7
1,2023,1,2,157.5,105.6,134.3
2,2023,1,3,103.8,88.3,91.5
3,2023,1,5,100.3,76.6,92.4
4,2023,1,6,176.3,127.5,177.8
...,...,...,...,...,...,...
505,2021,17,7,112.5,105.9,99.9
506,2021,17,9,162.8,161.6,156.6
507,2021,17,10,203.7,150.7,190.7
508,2021,17,11,128.2,113.8,128.5


In [103]:
# Repeat steps for first 3 years
league_id = 564698

yrs = [2020,2019,2018]
posns = ['QB', 'RB', 'WR', 'Flex', 'TE', 'D/ST','K']
struc = [1,2,2,1,1,1,1]

data = {}
slates = {}
ds = {}
print('Week:', end=' ')
for y in yrs:
    data[y] = {}   # Initialize the year-specific dictionary in data
    slates[y] = {} # Initialize the year-specific dictionary in slates
    ds[y] = {}
    for week in range(1,18):
        print(week, y, end=' ')
        d      = get_matchups(league_id, y, week, swid=swid, espn=espn_s2)
        wslate = get_slates(d, y, week)
        wdata  = compute_pts(wslate, posns, struc)
        data[y][week] = wdata
        slates[y][week] = wslate
        ds[y][week] = d

Week: 1 2020 2 2020 3 2020 4 2020 5 2020 6 2020 7 2020 8 2020 9 2020 10 2020 11 2020 12 2020 13 2020 14 2020 15 2020 16 2020 17 2020 1 2019 2 2019 3 2019 4 2019 5 2019 6 2019 7 2019 8 2019 9 2019 10 2019 11 2019 12 2019 13 2019 14 2019 15 2019 16 2019 17 2019 1 2018 2 2018 3 2018 4 2018 5 2018 6 2018 7 2018 8 2018 9 2018 10 2018 11 2018 12 2018 13 2018 14 2018 15 2018 16 2018 17 2018 

In [104]:
lineup_df1 = pd.DataFrame()

# Iterate through each year, week, and team DataFrame
for year, week_data in slates.items():
    for week, team_data in week_data.items():
        for team_id, df in team_data.items():
            # Add 'Year' and 'TeamID' columns to each DataFrame
            df['Year'] = year
            df['Week'] = week
            df['TeamID'] = team_id
            # Append to the combined DataFrame
            lineup_df1 = pd.concat([lineup_df1, df], ignore_index=True)

# Optionally, reorder columns if necessary
#lineup_df1

,Name,Year,Week,SlotID,Slot,Pos,Actual,Proj,TeamID
0,Dalvin Cook,2020,1,2,RB,RB,22.5,22.676799,1
1,Travis Kelce,2020,1,6,TE,TE,14.0,15.691530,1
2,Kenny Golladay,2020,1,20,Bench,WR,0.0,0.000000,1
3,Robert Woods,2020,1,4,WR,WR,21.0,13.185041,1
4,Le'Veon Bell,2020,1,2,RB,RB,6.2,13.779261,1
...,...,...,...,...,...,...,...,...,...
8303,DJ Moore,2018,17,20,Bench,WR,8.5,7.312238,12
8304,Vance McDonald,2018,17,20,Bench,TE,4.2,6.165555,12
8305,Jalen Richard,2018,17,20,Bench,RB,2.3,8.987427,12
8306,Derrick Henry,2018,17,2,RB,RB,12.3,18.547673,12


In [107]:
df_lineups_all = pd.concat([lineup_df,lineup_df1])

In [108]:
df2 = pd.DataFrame.from_dict(
    {(year, week, team): stats
     for year, weeks in data.items()
     for week, teams in weeks.items()
     for team, stats in teams.items()},
    orient='index'
)

df2.reset_index(inplace=True)
df2.columns = ['Year', 'Week', 'TeamID', 'Opts', 'Epts', 'Apts']



,Year,Week,TeamID,Opts,Epts,Apts
0,2020,1,1,159.4,148.8,148.8
1,2020,1,2,135.5,114.9,115.1
2,2020,1,3,182.0,171.6,171.6
3,2020,1,5,162.8,159.6,159.6
4,2020,1,6,138.6,115.4,127.5
...,...,...,...,...,...,...
505,2018,17,7,123.9,114.3,108.5
506,2018,17,9,115.0,74.1,78.9
507,2018,17,10,109.1,85.1,77.1
508,2018,17,11,125.4,122.0,88.0


In [109]:
df_pts = pd.concat([df1,df2])

In [214]:
df_pts.to_csv('df_pts.csv')